Completed:
- Extracted data at intervals of first record each day
- Filtered data from tank NDS002
- Created calculation model for NPK ratio
- Push Code to GitHub

Remaining Task:
- Categorize into levels
- Model to extract time taken to reach ‘Optimal’ level
- Automate NPK calculation model for future data
- Test on increased size of dataset

In [238]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Extracted data from NDS002:

import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="smart_composting_api",
    user="npds_2025",
    password="npds_2025_eh0atNiA5MVx2FYY3UnqVo5Vzv_0N9MRnSZ_3dkJgT_r2EIONEpFzV1o3IXHSFsjUX8hXT-9OgKqt8f512RPWJohKdM_pA-dAfimfXXOuke5C0Z9irOt4GrEV5R",
    host="db.composting.tinkerthings.global",
    port="6969"
)

# Define Custom SQL Query from PgAdmin
# Extract Sensor Data from tank NDS002 only
query = """
WITH RankedData AS (
  SELECT 
    SD.dataid, 
    D.devicename, 
    S.sensor, 
    S.sensorid, 
    SD.value, 
    DD.dbtimestamp,
    ROW_NUMBER() OVER (
      PARTITION BY 
        D.devicename, 
        S.sensorid, 
        DATE(DD.dbtimestamp)
      ORDER BY 
        DD.dbtimestamp ASC
    ) AS rn
  FROM public.sensordata AS SD
  JOIN public.devicedata   DD ON SD.dataid   = DD.dataid
  JOIN public.sensors      S  ON S.sensorid  = SD.sensorid
  JOIN public.devices      D  ON D.deviceid  = DD.deviceid
  WHERE D.devicedescription LIKE 'NP DS 2024 Group A%'
  OR D.devicedescription LIKE 'NP DS 2024 Group D%'
  OR D.devicedescription LIKE 'NP DS 2024 Group E%'
  OR D.devicename = 'NDS006'
  OR D.devicename = 'NDS011'
  OR D.devicename = 'NDS012'
  OR D.devicename = 'NDS013'
  OR D.devicename = 'NDS015'
)
SELECT 
  dataid, 
  devicename, 
  sensor, 
  sensorid, 
  value, 
  dbtimestamp
FROM RankedData
WHERE rn = 1
ORDER BY dbtimestamp DESC;
"""

# Load result into DataFrame
chunks = pd.read_sql_query(query, conn, chunksize=1000)

# Combine or process each chunk
df_list = []
for chunk in chunks:
    print(f"Loaded chunk with {len(chunk)} rows.")
    df_list.append(chunk)  # Or process chunk directly

# Combine all chunks into a full DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Total rows loaded: {len(df)}")

# Close connection
conn.close()

df.to_csv("Raw Data.csv ", index=False)

In [239]:
# load data
df = pd.read_csv('Raw Data.csv')

In [240]:
df.head(10)

,dataid,devicename,sensor,sensorid,value,dbtimestamp
0,71267447,NDS016,Soil Nitrogen,12,13.0,2025-05-28 00:00:48.939404
1,71267443,NDS016,Soil Moisture,9,18.1,2025-05-28 00:00:48.939404
2,71267448,NDS016,Soil Phosphorus,13,76.0,2025-05-28 00:00:48.939404
3,71267449,NDS016,Soil Potassium,14,68.0,2025-05-28 00:00:48.939404
4,71267446,NDS016,Soil pH,10,5.5,2025-05-28 00:00:48.939404
5,71267444,NDS016,Soil Temperature,8,26.5,2025-05-28 00:00:48.939404
6,71267445,NDS016,Soil EC,11,247.0,2025-05-28 00:00:48.939404
7,71119631,NDS016,Soil Temperature,8,27.3,2025-05-27 00:00:19.025437
8,71119630,NDS016,Soil Moisture,9,19.2,2025-05-27 00:00:19.025437
9,71119636,NDS016,Soil Potassium,14,80.0,2025-05-27 00:00:19.025437


In [241]:
df['dbtimestamp'] = df['dbtimestamp'].astype('datetime64[ns]')

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10909 entries, 0 to 10908
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dataid       10909 non-null  int64         
 1   devicename   10909 non-null  object        
 2   sensor       10909 non-null  object        
 3   sensorid     10909 non-null  int64         
 4   value        10909 non-null  float64       
 5   dbtimestamp  10909 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 511.5+ KB


Filter out 1970

In [243]:
# Filter out rows where the year is 2017
filtered_df = df[df["dbtimestamp"].dt.year != 1970]

filtered_df.tail()

,dataid,devicename,sensor,sensorid,value,dbtimestamp
10904,381388,NDS016,Soil Nitrogen,12,255.0,2024-07-18 11:50:40.977031
10905,381387,NDS016,Soil pH,10,0.0,2024-07-18 11:50:40.968742
10906,381386,NDS016,Soil EC,11,0.0,2024-07-18 11:50:40.964882
10907,381385,NDS016,Soil Temperature,8,1408.0,2024-07-18 11:50:40.961036
10908,381384,NDS016,Soil Moisture,9,1152.0,2024-07-18 11:50:40.952504


Filter out npk level < 1000

In [244]:
# Filter rows where Age > 25 and Name starts with 'B'
filtered_df = filtered_df[(filtered_df['value'] < 2000) & (filtered_df['value'] > 0)]

Feature Engineering: Number of Days since First Recording

In [245]:
filtered_df['dbtimestamp'] = filtered_df['dbtimestamp'].astype('datetime64[ns]')

In [246]:
filtered_df['days_from_start'] = (filtered_df['dbtimestamp'] - filtered_df['dbtimestamp'].min()).dt.days

In [247]:
filtered_df.head()

,dataid,devicename,sensor,sensorid,value,dbtimestamp,days_from_start
0,71267447,NDS016,Soil Nitrogen,12,13.0,2025-05-28 00:00:48.939404,313
1,71267443,NDS016,Soil Moisture,9,18.1,2025-05-28 00:00:48.939404,313
2,71267448,NDS016,Soil Phosphorus,13,76.0,2025-05-28 00:00:48.939404,313
3,71267449,NDS016,Soil Potassium,14,68.0,2025-05-28 00:00:48.939404,313
4,71267446,NDS016,Soil pH,10,5.5,2025-05-28 00:00:48.939404,313


Format dataframe so each sensor becomes column and value is the cell content

In [248]:
# Pivot with both dbtimestamp and devicename in the index
npk_row = filtered_df.pivot(index=['dbtimestamp', 'devicename'], columns='sensor', values='value').reset_index()

# Sort by dbtimestamp (and optionally by devicename)
npk_row = npk_row.sort_values(by='dbtimestamp', ascending=False).reset_index(drop=True)

# Display result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,2025-05-28 00:00:48.939404,NDS016,NaN,NaN,NaN,247.0,18.1,13.0,76.0,68.0,26.5,5.5
1,2025-05-27 00:00:19.025437,NDS016,NaN,NaN,NaN,270.0,19.2,17.0,87.0,80.0,27.3,5.5
2,2025-05-26 00:01:19.007408,NDS016,NaN,NaN,NaN,292.0,20.7,22.0,98.0,90.0,26.9,5.2
3,2025-05-25 00:00:44.393652,NDS016,NaN,NaN,NaN,318.0,21.1,27.0,110.0,103.0,27.1,4.8
4,2025-05-24 00:01:50.268697,NDS016,NaN,NaN,NaN,348.0,19.3,34.0,125.0,117.0,27.1,4.9


Categorize into Numerical Columns

In [249]:
import numpy as np
import pandas as pd

num_cols = [
    n for n in npk_row.columns
    if npk_row[n].dtypes != 'O' and not np.issubdtype(npk_row[n].dtype, np.datetime64)
]

df_num = npk_row[num_cols]

df_num.head()


sensor,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,NaN,NaN,NaN,247.0,18.1,13.0,76.0,68.0,26.5,5.5
1,NaN,NaN,NaN,270.0,19.2,17.0,87.0,80.0,27.3,5.5
2,NaN,NaN,NaN,292.0,20.7,22.0,98.0,90.0,26.9,5.2
3,NaN,NaN,NaN,318.0,21.1,27.0,110.0,103.0,27.1,4.8
4,NaN,NaN,NaN,348.0,19.3,34.0,125.0,117.0,27.1,4.9


Null Imputation: Iterative Imputer

In [250]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

# Apply Iterative Imputer
imp = IterativeImputer(max_iter=10, random_state=0)
npk_row[num_cols] = imp.fit_transform(npk_row[num_cols])

c:\Users\leong\anaconda\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [251]:
print(npk_row.isnull().mean().sort_values(ascending = False))

sensor
dbtimestamp         0.0
devicename          0.0
Air Temperature     0.0
CO2                 0.0
Humidity            0.0
Soil EC             0.0
Soil Moisture       0.0
Soil Nitrogen       0.0
Soil Phosphorus     0.0
Soil Potassium      0.0
Soil Temperature    0.0
Soil pH             0.0
dtype: float64


In [252]:
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,2025-05-28 00:00:48.939404,NDS016,2.160823e+13,2.540875e+14,-3.724656e+14,247.0,18.1,13.0,76.0,68.0,26.5,5.5
1,2025-05-27 00:00:19.025437,NDS016,2.058217e+13,2.423452e+14,-3.551665e+14,270.0,19.2,17.0,87.0,80.0,27.3,5.5
2,2025-05-26 00:01:19.007408,NDS016,1.918968e+13,2.266295e+14,-3.319457e+14,292.0,20.7,22.0,98.0,90.0,26.9,5.2
3,2025-05-25 00:00:44.393652,NDS016,1.858465e+13,2.192980e+14,-3.212630e+14,318.0,21.1,27.0,110.0,103.0,27.1,4.8
4,2025-05-24 00:01:50.268697,NDS016,1.920255e+13,2.249196e+14,-3.299639e+14,348.0,19.3,34.0,125.0,117.0,27.1,4.9


Create Calculation Method for NPK:
- Automate for future data

In [253]:
def calculate_npk_ratio(row):
    N = row['Soil Nitrogen']
    P = row['Soil Phosphorus']
    K = row['Soil Potassium']

    # Avoid division by zero
    if min(N, P, K) == 0:
        return float('nan')

    # Normalize based on smallest value
    min_val = min(N, P, K)
    return f"{round(N/min_val, 1)} : {round(P/min_val, 1)} : {round(K/min_val, 1)}"

In [254]:
# Create new column
npk_row['NPK Ratio'] = npk_row.apply(calculate_npk_ratio, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio
0,2025-05-28 00:00:48.939404,NDS016,2.160823e+13,2.540875e+14,-3.724656e+14,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2
1,2025-05-27 00:00:19.025437,NDS016,2.058217e+13,2.423452e+14,-3.551665e+14,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7
2,2025-05-26 00:01:19.007408,NDS016,1.918968e+13,2.266295e+14,-3.319457e+14,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1
3,2025-05-25 00:00:44.393652,NDS016,1.858465e+13,2.192980e+14,-3.212630e+14,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8
4,2025-05-24 00:01:50.268697,NDS016,1.920255e+13,2.249196e+14,-3.299639e+14,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4


Categorize into 'Low', 'High', 'Optimal'
- Long Bean: 1:2:2
- Spinach: 2:1:1
- Lady Finger 1:1:1

In [255]:
def nitrogen(value):
    if value <= 100:
        return 'Low'
    elif 100 < value <= 300:
        return 'High'
    else:
        return 'Optimal'

In [256]:
def pp(value):
    if value <= 300:
        return 'Low'
    elif 300 < value <= 600:
        return 'High'
    else:
        return 'Optimal'

In [257]:
npk_row['N Category'] = npk_row['Soil Nitrogen'].apply(nitrogen)
npk_row['P Category'] = npk_row['Soil Phosphorus'].apply(pp)
npk_row['K Category'] = npk_row['Soil Potassium'].apply(pp)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category
0,2025-05-28 00:00:48.939404,NDS016,2.160823e+13,2.540875e+14,-3.724656e+14,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low
1,2025-05-27 00:00:19.025437,NDS016,2.058217e+13,2.423452e+14,-3.551665e+14,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low
2,2025-05-26 00:01:19.007408,NDS016,1.918968e+13,2.266295e+14,-3.319457e+14,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low
3,2025-05-25 00:00:44.393652,NDS016,1.858465e+13,2.192980e+14,-3.212630e+14,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low
4,2025-05-24 00:01:50.268697,NDS016,1.920255e+13,2.249196e+14,-3.299639e+14,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low


In [258]:
npk_row.tail()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category
4953,2024-07-18 11:58:00.154610,NDS005,-5.075899e+12,-5.001031e+13,7.589463e+13,2362.680776,10.400000,106.085291,369.393764,367.861975,-42.556610,13.144413,1.0 : 3.5 : 3.5,High,High,High
4954,2024-07-18 11:50:40.984922,NDS016,3.377078e+13,3.050702e+14,-4.715412e+14,-5257.081620,62.873624,923.543770,376.728697,259.000000,193.113784,-25.094527,3.6 : 1.5 : 1.0,Optimal,High,Low
4955,2024-07-18 11:50:40.977031,NDS016,-8.897358e+11,-1.699542e+13,2.323837e+13,582.592640,64.184189,255.000000,382.427770,370.407422,122.438611,14.813145,1.0 : 1.5 : 1.5,High,High,High
4956,2024-07-18 11:50:40.961036,NDS016,4.429516e+13,4.415231e+14,-6.684760e+14,-13164.384582,461.535027,606.939078,367.650413,356.218431,1408.000000,43.525111,1.7 : 1.0 : 1.0,Optimal,High,High
4957,2024-07-18 11:50:40.952504,NDS016,1.178499e+14,1.171439e+15,-1.774588e+15,-37298.354351,1152.000000,1381.526624,370.699927,338.855949,3608.912971,82.070173,4.1 : 1.1 : 1.0,Optimal,High,High


In [259]:
# Long Bean: 1:2:2
# Spinach: 2:1:1
# Lady Finger 1:1:1

def categorize_npk(row):
    ratio = row['NPK Ratio']

    if ratio == '1.0 : 2.0 : 2.0':
        return 'Long Bean'
    elif ratio == '2.0 : 1.0 : 1.0':
        return 'Spinach'
    elif ratio == '1.0 : 1.0 : 1.0':
        return 'Lady Finger'
    else:
        return 'Invalid'

In [260]:
npk_row['NPK Level'] = npk_row.apply(categorize_npk, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level
0,2025-05-28 00:00:48.939404,NDS016,2.160823e+13,2.540875e+14,-3.724656e+14,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid
1,2025-05-27 00:00:19.025437,NDS016,2.058217e+13,2.423452e+14,-3.551665e+14,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid
2,2025-05-26 00:01:19.007408,NDS016,1.918968e+13,2.266295e+14,-3.319457e+14,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid
3,2025-05-25 00:00:44.393652,NDS016,1.858465e+13,2.192980e+14,-3.212630e+14,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low,Invalid
4,2025-05-24 00:01:50.268697,NDS016,1.920255e+13,2.249196e+14,-3.299639e+14,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low,Invalid


In [261]:
def npk_done(row):
    ratio = row['NPK Ratio']
    n = row['N Category']
    p = row['P Category']
    k = row['K Category']

    if ratio == '1.0 : 2.0 : 2.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Long Bean Done'
    elif ratio == '2.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Spinach Done'
    elif ratio == '1.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Lady Finger Done'
    elif n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Optimal NPK'
    else:
        return 'Not Done Composting'

In [262]:
npk_row['NPK Done'] = npk_row.apply(npk_done, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-28 00:00:48.939404,NDS016,2.160823e+13,2.540875e+14,-3.724656e+14,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-27 00:00:19.025437,NDS016,2.058217e+13,2.423452e+14,-3.551665e+14,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-26 00:01:19.007408,NDS016,1.918968e+13,2.266295e+14,-3.319457e+14,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-25 00:00:44.393652,NDS016,1.858465e+13,2.192980e+14,-3.212630e+14,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-24 00:01:50.268697,NDS016,1.920255e+13,2.249196e+14,-3.299639e+14,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low,Invalid,Not Done Composting


In [263]:
extract = npk_row.copy()

In [264]:
extract.count()

sensor
dbtimestamp         4958
devicename          4958
Air Temperature     4958
CO2                 4958
Humidity            4958
Soil EC             4958
Soil Moisture       4958
Soil Nitrogen       4958
Soil Phosphorus     4958
Soil Potassium      4958
Soil Temperature    4958
Soil pH             4958
NPK Ratio           4958
N Category          4958
P Category          4958
K Category          4958
NPK Level           4958
NPK Done            4958
dtype: int64

In [265]:
# Filter rows with null values
rows_with_nulls = extract[extract.isnull().any(axis=1)]

# Print rows with null values
rows_with_nulls.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done


In [266]:
# List of columns to exclude
exclude_cols = ['Air Temperature', 'CO2', 'Humidity']

# Select all columns except the excluded ones
df_filtered = extract.drop(columns=exclude_cols)

df_filtered.head()

sensor,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-25 00:00:44.393652,NDS016,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-24 00:01:50.268697,NDS016,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low,Invalid,Not Done Composting


In [267]:
# Apply value filters
df_filtered = df_filtered[
    (df_filtered['Soil EC'].between(0, 3000)) &
    (df_filtered['Soil Moisture'].between(0, 3000)) &
    (df_filtered['Soil Nitrogen'].between(0, 3000)) &
    (df_filtered['Soil Phosphorus'].between(0, 3000)) &
    (df_filtered['Soil Potassium'].between(0, 3000)) &
    (df_filtered['Soil Temperature'].between(20, 35)) &
    (df_filtered['Soil pH'].between(1, 12))
]

df_filtered.head()


sensor,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-25 00:00:44.393652,NDS016,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-24 00:01:50.268697,NDS016,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low,Invalid,Not Done Composting


In [268]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 975
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dbtimestamp       761 non-null    datetime64[ns]
 1   devicename        761 non-null    object        
 2   Soil EC           761 non-null    float64       
 3   Soil Moisture     761 non-null    float64       
 4   Soil Nitrogen     761 non-null    float64       
 5   Soil Phosphorus   761 non-null    float64       
 6   Soil Potassium    761 non-null    float64       
 7   Soil Temperature  761 non-null    float64       
 8   Soil pH           761 non-null    float64       
 9   NPK Ratio         761 non-null    object        
 10  N Category        761 non-null    object        
 11  P Category        761 non-null    object        
 12  K Category        761 non-null    object        
 13  NPK Level         761 non-null    object        
 14  NPK Done          761 non-null 

In [269]:
# Will not be using NDS002 for now

df_filtered.groupby('devicename').size()

devicename
NDS002      1
NDS004     98
NDS005     98
NDS006     98
NDS011     98
NDS012     98
NDS015     98
NDS016    172
dtype: int64

Connnect to NP Feeding Log

In [270]:
# Load the uploaded Excel file
excel_path = "[NgeeAnn] Feeding Logs.xlsx"

In [271]:
import pandas as pd

# Load Excel file
xls = pd.ExcelFile(excel_path)
sheet_names = xls.sheet_names

# Load the 3rd and 4th sheets (indexes 2 and 3)
df_third = xls.parse(sheet_names[2])
df_fourth = xls.parse(sheet_names[3])
df_fifth = xls.parse(sheet_names[4])

# Concatenate them (by default, axis=0, i.e., stack rows)
df_feeding_log = pd.concat([df_third, df_fourth, df_fifth], ignore_index=True)

# Preview the result
df_feeding_log.head()

,Date,NDS001,NDS002,NDS003,NDS004,NDS005,NDS006,NDS007,NDS008,NDS009,NDS010,NDS011,NDS012,NDS013,NDS014,NDS015,NDS016
0,2024-06-01,"70g of distilled water, 3g of dried radish top...","4g of fresh mint leaves, 5g of dried mango ski...","4g of dried radish tops, 6g of crushed eggshel...","50g of distilled water, 1g of dried radish top...","50g of distilled water, 6g of crushed eggshell...","3g of dried radish tops, 5g of dried mango skins","15g of coffee grounds, 3g of dried chye sim","3g of dried radish tops, 7g of crushed eggshel...","25g of coffee grounds, 4g of dried chye sim","5g of dried radish tops, 6g of crushed eggshel...","45g of distilled water, 5g of dried radish top...",NaN,NaN,NaN,NaN,NaN
1,2024-06-02,45g of distilled water,"45g of distilled water, 4g of dried chye sim","3g of dried radish tops, 8g of crushed eggshel...","8g of crushed eggshells, 10g of coffee grounds...","70g of distilled water, 15g of coffee grounds","40g of distilled water, 3g of dried radish top...","5g of dried radish tops, 15g of coffee grounds...","40g of distilled water, 4g of crushed eggshell...",45g of distilled water,2g of dried mango skins,"100g of distilled water, 5g of crushed eggshel...","50g of dried radish tops, 50g of dried chye sim","50g of dried radish tops, 50g of dried chye sim","50g of dried radish tops, 50g of dried chye sim",NaN,"50g of dried radish tops, 50g of dried chye sim"
2,2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-04,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water
4,2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filter Out Tanks:

1) NDS002      1
2) NDS004     98
3) NDS005     98
4) NDS006     98
5) NDS011     98
6) NDS012     98
7) NDS015     98
8) NDS016    172

In [272]:
df_feeding_log = df_feeding_log[['Date', 'NDS004', 'NDS005', 'NDS006', 'NDS011', 'NDS012', 'NDS015', 'NDS016']]

df_feeding_log.head()

,Date,NDS004,NDS005,NDS006,NDS011,NDS012,NDS015,NDS016
0,2024-06-01,"50g of distilled water, 1g of dried radish top...","50g of distilled water, 6g of crushed eggshell...","3g of dried radish tops, 5g of dried mango skins","45g of distilled water, 5g of dried radish top...",NaN,NaN,NaN
1,2024-06-02,"8g of crushed eggshells, 10g of coffee grounds...","70g of distilled water, 15g of coffee grounds","40g of distilled water, 3g of dried radish top...","100g of distilled water, 5g of crushed eggshel...","50g of dried radish tops, 50g of dried chye sim",NaN,"50g of dried radish tops, 50g of dried chye sim"
2,2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-04,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water
4,2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [273]:
convert_feedlog = df_feeding_log.melt(
    id_vars='Date',
    var_name='devicename',
    value_name='feeding_description'
)

# Preview the transformed feeding log
convert_feedlog.head()

,Date,devicename,feeding_description
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top..."
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds..."
2,2024-06-03,NDS004,NaN
3,2024-06-04,NDS004,200g of water
4,2024-06-05,NDS004,NaN


In [274]:
convert_feedlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128 entries, 0 to 2127
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 2128 non-null   datetime64[ns]
 1   devicename           2128 non-null   object        
 2   feeding_description  427 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 50.0+ KB


Green & Brown Categorization

In [275]:
# Load the first sheet, which contains the greens and browns reference list
df_greens_browns = pd.read_excel(excel_path, sheet_name=0, header=1)

# Display to inspect
df_greens_browns.head()

,S/N,Food Name,Emissions (kg CO2/kg),Category,pH Value
0,1,Bread crusts,0.14,Brown,Neutral
1,2,Cauliflower leaves,0.07,Brown,Acidic
2,3,Cheese rinds,1.20,Brown,Acidic
3,4,Coffee grounds,0.20,Brown,Acidic
4,5,Ground coffee,0.23,Brown,Acidic


In [276]:
df_greens_browns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   S/N                    84 non-null     int64  
 1   Food Name              84 non-null     object 
 2   Emissions (kg CO2/kg)  84 non-null     float64
 3   Category               84 non-null     object 
 4   pH Value               84 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.4+ KB


Add brown & green categorization, feeding description

In [277]:
# Step 2: Normalize food name reference
df_greens_browns['Food Name'] = df_greens_browns['Food Name'].str.lower().str.strip()
df_greens_browns['Category'] = df_greens_browns['Category'].str.strip()
food_to_category = dict(zip(df_greens_browns['Food Name'], df_greens_browns['Category']))

In [278]:
# Step 3: Function to classify feeding description
def categorize_feeding(desc):
    if pd.isnull(desc):
        return 'None'
    desc = desc.lower()
    matches = [food_to_category[food] for food in food_to_category if food in desc]
    if not matches:
        return 'Unknown'
    return matches[0] if all(x == matches[0] for x in matches) else 'Mixed'

In [279]:
# Step 4: Apply feeding category classification to feeding log
convert_feedlog['Feeding Category'] = convert_feedlog['feeding_description'].apply(categorize_feeding)

In [280]:
convert_feedlog.head()

,Date,devicename,feeding_description,Feeding Category
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top...",Green
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds...",Mixed
2,2024-06-03,NDS004,NaN,None
3,2024-06-04,NDS004,200g of water,Unknown
4,2024-06-05,NDS004,NaN,None


In [281]:
# Renaming columns
convert_feedlog.rename(columns={'Date': 'dbtimestamp'}, inplace=True)
convert_feedlog.head()

,dbtimestamp,devicename,feeding_description,Feeding Category
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top...",Green
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds...",Mixed
2,2024-06-03,NDS004,NaN,None
3,2024-06-04,NDS004,200g of water,Unknown
4,2024-06-05,NDS004,NaN,None


In [282]:
df_combined = pd.merge(
    df_filtered.assign(date=pd.to_datetime(df_filtered['dbtimestamp']).dt.date),
    convert_feedlog.assign(date=pd.to_datetime(convert_feedlog['dbtimestamp']).dt.date)[['devicename', 'date', 'feeding_description', 'Feeding Category']],
    on=['devicename', 'date'],
    how='left'
)

In [283]:
df_combined = df_combined.drop('date', axis=1)
df_combined.head()

,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done,feeding_description,Feeding Category
0,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
1,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
2,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
3,2025-05-25 00:00:44.393652,NDS016,318.0,21.1,27.0,110.0,103.0,27.1,4.8,1.0 : 4.1 : 3.8,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
4,2025-05-24 00:01:50.268697,NDS016,348.0,19.3,34.0,125.0,117.0,27.1,4.9,1.0 : 3.7 : 3.4,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN


Extract Data

In [284]:
df_combined.to_csv("2024 Extracted.csv ", index=False)